In [1]:
from datetime import time

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_mantine_components as dmc

from Utils import City, AggregationLevel, TrafficType
from DataPreprocessing.WeatherData.Data import WeatherData
from DataPreprocessing.TrafficData.Data import CityTrafficData
from DataPreprocessing.GeoData.GeoData import IrisGeoData
from ExpectedBedTime.ExpectedBedTimeAPI import ExpectedBedTimeAPI
from DataPreprocessing.AdminData.AdminDataComplete import AdminData
from ExpectedBedTime.Plots import StartBedTimeRobustness
from FeatureSelection.BasicFeatureImportanceCalculator import BasicFeatureImportanceCalculator
from FeatureSelection.RegressionConstructor import Regression, FeatureConstructor
from DataPreprocessing.GeoData.GeoDataComplete import GeoData, GeoDataType

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
admin_data = AdminData()

In [181]:
geo_data = GeoData()
geo_data.load(GeoDataType.IRIS)

In [9]:
traffic_data = (CityTrafficData(city=city, geo_data_type=GeoDataType.IRIS, traffic_type=TrafficType.USERS) for city in City)

In [10]:
expected_bed_time = ExpectedBedTimeAPI.compute_expected_bed_time(traffic_data=traffic_data)

100%|██████████| 77/77 [00:15<00:00,  5.06it/s]


# EXPLORATORY DATA ANALYSIS

In [400]:
iris_subset = expected_bed_time.data.index

In [401]:
feature_constructor = FeatureConstructor(admin_data=admin_data, geo_data=geo_data)

## Variables considered

In [402]:
iris_centrality = feature_constructor.iris_centrality(iris=iris_subset)

In [403]:
iris_density_of_city = feature_constructor.iris_density_of_city(iris=iris_subset).rename(columns={'density': 'density_of_city'})

In [404]:
iris_pop_density = feature_constructor.iris_density(iris=iris_subset, var_names=['P19_POP']).rename(columns={'P19_POP': 'POP_DEN'})

In [405]:
iris_business_density = feature_constructor.iris_business_density(iris=iris_subset)

In [406]:
age_shares = feature_constructor.iris_shares(iris=iris_subset, var_names=['P19_POP1529', 'P19_POP3044', 'P19_POP4559', 'P19_POP6074', 'P19_POP75P'])

In [407]:
lonely_shares = feature_constructor.iris_shares(iris=iris_subset, var_names=['P19_POP15P_PSEUL'])

In [408]:
income = admin_data.get_admin_data(subset=iris_subset)[['DEC_MED19', 'DEC_GI19']]

In [409]:
shares_of_workers_with_far_work = feature_constructor.iris_shares(iris=iris_subset, var_names=['P19_ACTOCC15P_ILT2P'])

In [410]:
education = feature_constructor.iris_shares(iris=iris_subset, var_names=['P19_ACT_DIPLMIN','P19_ACT_BAC', 'P19_ACT_SUP2'])

In [411]:
unemployment = feature_constructor.iris_shares(iris=iris_subset, var_names=['P19_CHOM1524', 'P19_CHOM2554', 'P19_CHOM5564'])

In [412]:
retirees = feature_constructor.iris_shares(iris=iris_subset, var_names=['P19_RETR1564'])

In [413]:
nationality = feature_constructor.iris_shares(iris=iris_subset, var_names=['P19_POP_ETR'])

In [414]:
house_sizes = feature_constructor.iris_shares(iris=iris_subset, var_names=['P19_RP_M30M2', 'P19_RP_3040M2', 'P19_RP_4060M2', 'P19_RP_6080M2', 'P19_RP_80100M2', 'P19_RP_100120M2', 'P19_RP_120M2P'])

In [415]:
men = feature_constructor.iris_shares(iris=iris_subset, var_names=['P19_POPH'])

In [416]:
families = feature_constructor.iris_shares(iris=iris_subset, var_names=['P19_POP15P_MARIEE', 'C19_FAM'])

In [417]:
# services = feature_constructor.iris_shares(iris=iris_subset, var_names=['EQUIP_E107', 'EQUIP_E108', 'EQUIP_E109', 'EQUIP_A504'])

In [418]:
displacement_to_work = feature_constructor.iris_shares(iris=iris_subset, var_names=['C19_ACTOCC15P_MAR', 'C19_ACTOCC15P_TCOM', 'C19_ACTOCC15P_PAS', 'C19_ACTOCC15P_VELO', 'C19_ACTOCC15P_VOIT'])

In [419]:
ebt = expected_bed_time.data['mean_float'].to_frame(name='expected_bed_time')

## Merge variables

In [420]:
data = pd.merge(iris_pop_density, ebt, left_index=True, right_index=True, how='inner')
data = pd.merge(data, iris_density_of_city, left_index=True, right_index=True, how='inner')
data = pd.merge(data, iris_centrality, left_index=True, right_index=True, how='inner')
data = pd.merge(data, income, left_index=True, right_index=True, how='inner')
data = pd.merge(data, iris_business_density + np.quantile(iris_business_density, 0.1), left_index=True, right_index=True, how='inner')
data = np.log(data)
data = pd.merge(data, age_shares, left_index=True, right_index=True, how='inner')
data = pd.merge(data, lonely_shares, left_index=True, right_index=True, how='inner')
data = pd.merge(data, shares_of_workers_with_far_work, left_index=True, right_index=True, how='inner')
data = pd.merge(data, education, left_index=True, right_index=True, how='inner')
data = pd.merge(data, unemployment, left_index=True, right_index=True, how='inner')
data = pd.merge(data, retirees, left_index=True, right_index=True, how='inner')
data = pd.merge(data, nationality, left_index=True, right_index=True, how='inner')
data = pd.merge(data, house_sizes, left_index=True, right_index=True, how='inner')
data = pd.merge(data, men, left_index=True, right_index=True, how='inner')
data = pd.merge(data, families, left_index=True, right_index=True, how='inner')
data = pd.merge(data, displacement_to_work, left_index=True, right_index=True, how='inner')
data.dropna(inplace=True)
data.shape, np.array(data.columns)

((5758, 37),
 array(['POP_DEN', 'expected_bed_time', 'density_of_city', 'centrality',
        'DEC_MED19', 'DEC_GI19', 'business_density', 'P19_POP1529',
        'P19_POP3044', 'P19_POP4559', 'P19_POP6074', 'P19_POP75P',
        'P19_POP15P_PSEUL', 'P19_ACTOCC15P_ILT2P', 'P19_ACT_DIPLMIN',
        'P19_ACT_BAC', 'P19_ACT_SUP2', 'P19_CHOM1524', 'P19_CHOM2554',
        'P19_CHOM5564', 'P19_RETR1564', 'P19_POP_ETR', 'P19_RP_M30M2',
        'P19_RP_3040M2', 'P19_RP_4060M2', 'P19_RP_6080M2',
        'P19_RP_80100M2', 'P19_RP_100120M2', 'P19_RP_120M2P', 'P19_POPH',
        'P19_POP15P_MARIEE', 'C19_FAM', 'C19_ACTOCC15P_MAR',
        'C19_ACTOCC15P_TCOM', 'C19_ACTOCC15P_PAS', 'C19_ACTOCC15P_VELO',
        'C19_ACTOCC15P_VOIT'], dtype=object))

## Run analysis and plot

In [421]:
x_axis = 'POP_DEN'
features = data[[c for c in data.columns if c not in ['expected_bed_time', 'density_of_city']]]
labels = data['expected_bed_time'].to_frame()
regression = Regression(features=features, labels=labels)
regression.plot(x_axis=x_axis)